In [1]:
############################
# CS-340 Project 2
############################
# Author:  Chris Sharrock
#          code base provided by SNHU
# Version: 1.0
# Date Last Modified: 10/14/23
#
# This program was written for Grazioso Salvare to assist with finding animals suitable for their training programs.
# It uses the AnimalShelter CRUD interface module to connect to and interact with the databases of local animal 
# shelters. The page created by this program offers a selection of radio buttons that tell the program which query 
# to run to find animals that are uniquely suitable to a specific type of rescue training. The page also features a
# geolocation map which shows where specific animals are being kept and a pie chart breaking down the numbers of 
# suitable animals by breed. The Dash framework can be used to add additional features as required.

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter CRUD interface module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Username and password for the database 
username = "aacuser"
password = "letMeIn!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Open and encode Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the page layout
app.layout = html.Div([

    # Display the title centered at the top of the page
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(   
    # Display the Grazioso Salvare logo centered under the page title
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())))
    ),
    html.Hr(),
    # Display the custom tag for the assignment
    html.Center(html.H1('site created by Chris Sharrock')),
    html.Hr(),
    # Display radio buttons for filtering by rescue discipline
    # Additional buttons can be added here for more types
    html.Div([
    dcc.RadioItems(
        id='filter-type',
        options=['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'], 
        value='Reset' # selected by default
        ),
    ]),
    # Display the query results in a table: can be sorted and filtered
    html.Div([
       # Define table properties
        dash_table.DataTable(id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
        
            ############################
            # Additional Table Settings
            ############################
            # Row Selection
            row_selectable='single', # allows rows to be selected one at a time (used for geolocation map below)
            selected_rows=[0],       # sets first row as selected by default (prevents an error due to no rows 
                                     # being selected)

            # Column Selection
            selected_columns=[], # an empty (but non-null) array for the update_styles function; prevents an error, but
                                 # can also be populated with column headers to be highlighted

            # Sorting and Filtering
            sort_action='native',   # allows sorting
            sort_mode='multi',      # allows multiple sorting criteria to be used at once
            filter_action='native', # allows filtering

            # Pagination
            page_current=0, # sets default starting page to first page
            page_size=10    # limits rows per page to 5
        ),
        html.Br(),
        html.Hr(),
        #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
            style={'display' : 'flex'},
                children=[
                    # Define geolocation map properties
                    html.Div(
                        id='map-id',
                        className='col s12 m6',
                    ),
                    # Define pie chart properties
                    html.Div(
                        id='graph-id',
                        className='col s12 m6',
                    )
                ])
        
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Updates the table in response to user input
@app.callback(Output('datatable-id','data'),    # returns the query results
              Output('datatable-id','columns'), # returns the column headers
              [Input('filter-type', 'value')])  # specifies to import the state of the radio button menu
def update_dashboard(value):

    ##################################################################
    # Select the appropriate query based on the selected radio button
    ##################################################################
    # Water Rescue Dogs:
    #     Breeds: Labrador Retriever Mix, Chesapeake Bay Retriever Mix, Newfoundland
    #     Sex:    Intact Female
    #     Age:    26-156 weeks
    if value == "Water Rescue":
        query = {"animal_type": "Dog",
                 "breed": 
                     {"$in": [
                         "Labrador Retriever Mix",
                         "Chesa Bay Retr Mix",
                         "Newfoundland"
                     ]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks":
                     {"$gte": 26,
                      "$lte": 156}
                }
    # Mountain Rescue Dogs:
    #     Breeds: German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler
    #     Sex:    Intact Male
    #     Age:    26-156 weeks
    elif value == "Mountain Rescue":
        query = {"animal_type": "Dog",
                 "breed": 
                     {"$in": [
                         "German Shepherd",
                         "Alaskan Malamute",
                         "Old English Sheepdog",
                         "Siberian Husky",
                         "Rottweiler"
                     ]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks":
                     {"$gte": 26,
                      "$lte": 156}
                }
    # Disaster Rescue Dogs:
    #     Breeds: Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler
    #     Sex:    Intact Male
    #     Age:    20-300 weeks
    elif value == "Disaster Rescue":
        query = {"animal_type": "Dog",
                 "breed": 
                     {"$in": [
                         "Doberman Pinscher",
                         "German Shepherd", 
                         "Golden Retriever",
                         "Bloodhound", 
                         "Rottweiler"
                     ]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks":
                     {"$gte": 20,
                      "$lte": 300}
                }    
    # Reset: sets query for all the animals in the database
    else:
        query = {}

    # Execute the query via the CRUD module
    df = pd.DataFrame.from_records(db.read(query))
        
    # MongoDB v5+ is going to return the '_id' column and that is going to have an 
    # invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
    # it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
    # inplace=True - it will return a new dataframe that does not contain the dropped column(s)
    df.drop(columns=['_id'],inplace=True)

    # Get the column names and convert the query results into a dict
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    # Return the query data and the column headers
    return (data,columns)


# Display the breeds of animal based on quantity represented in
# the data table as a pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")], # Data
    [Input('filter-type', 'value')])
def update_graphs(viewData, value):

    # If no training discipline is selected display a message instead of the chart
    # (chart of all animals in the database is unreadable)
    if value == 'Reset':
        return "No training discipline selected\n"
    
    # Pull the data from the data table
    dff = pd.DataFrame.from_dict(viewData)
    
    # Display a pie chart breaking down suitable animals by breed
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Suitable Animals by Breed')
        )]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    
    # If there is no data in the table, return without updating the map
    if viewData is None:
        return
    elif index is None:
        return
    
    # Pull the data from the data table
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is not None:
        row = index[0]
    else:
        return # returns without changing the map

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Start the server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:20043/
